In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import random
import pickle
import torch
import pandas as pd
import seaborn as sns
from os import listdir
from os.path import isfile, join, exists
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import shap
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from pathlib import Path
from pytorch_grad_cam import GradCAM

sys.path.append(os.path.join(os.path.abspath(''), '../..'))
from project.dataloaders.prna_dataloader import PrnaDataLoader
from project.models.prna import CTN
from project.utils.waveform_utils import WAVEFORM_SAMPLE_RATES, apply_filter, normalize
from project.utils.runners import train_mlp, test_mlp_single_fold, test_seq_mlp

pd.set_option('display.max_columns', None)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import io
import json
import sys
import os
import math

import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import sklearn
import pickle
import biosppy

# Transformer parameters
# Copied from the PRNA model
#
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
d_model = 256   # embedding size
nhead = 8       # number of heads
d_ff = 2048     # feed forward layer size
num_layers = 8  # number of encoding layers
dropout_rate = 0.2
model_name = 'ctn'
nb_demo = 2
nb_feats = 20
classes = sorted(['270492004', '164889003', '164890007', '426627000', '713427006',
                  '713426002', '445118002', '39732003', '164909002', '251146004',
                  '698252002', '10370003', '284470004', '427172004', '164947007',
                  '111975006', '164917005', '47665007', '427393009',
                  '426177001', '426783006', '427084000', '164934002',
                  '59931005'])

concept_to_desc = {
    "270492004": "first degree atrioventricular block",
    "164889003": "atrial fibrillation",
    "426627000": "bradycardia",
    "164890007": "atrial flutter",
    "713427006": "complete right bundle branch block",
    "713426002": "incomplete right bundle branch block",
    "445118002": "left anterior fascicular block",
    "39732003": "left axis deviation",
    "164909002": "left bundle branch block",
    "251146004": "low QRS voltage",
    "698252002": "non-specific intraventricular conduction delay",
    "10370003": "Pacing rhythm",
    "284470004": "Premature atrial contraction",
    "427172004": "Premature ventricular contractions",
    "164947007": "Prolonged PR interval",
    "111975006": "Prolonged QT interval",
    "164917005": "Q wave abnormal",
    "47665007": "Right axis deviation",
    "427393009": "Sinus arrhythmia",
    "426177001": "Sinus bradycardia",
    "426783006": "Sinus rhythm",
    "427084000": "Sinus tachycardia",
    "164934002": "T wave abnormal",
    "59931005": "T wave inversion",
    "59118001": "Right bundle branch block (disorder)",
    "63593006": "Supraventricular premature beats",
    "17338001": "Ventricular premature beats"
}

class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # Add position encodings to embeddings
        # x: embedding vects, [B x L x d_model]
        x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)
        return self.dropout(x)


class Transformer(nn.Module):
    '''
    Transformer encoder processes convolved ECG samples
    Stacks a number of TransformerEncoderLayers
    '''

    def __init__(self, d_model, h, d_ff, num_layers, dropout):
        super(Transformer, self).__init__()
        self.d_model = d_model
        self.h = h
        self.d_ff = d_ff
        self.num_layers = num_layers
        self.dropout = dropout
        self.pe = PositionalEncoding(d_model, dropout=0.1)

        encode_layer = nn.TransformerEncoderLayer(
            d_model=self.d_model,
            nhead=self.h,
            dim_feedforward=self.d_ff,
            dropout=self.dropout)
        self.transformer_encoder = nn.TransformerEncoder(encode_layer, self.num_layers)

    def forward(self, x):
        out = x.permute(0, 2, 1)
        out = self.pe(out)
        out = out.permute(1, 0, 2)
        out = self.transformer_encoder(out)
        out = out.mean(0)  # global pooling
        return out


# 15 second model
class CTN(nn.Module):
    def __init__(self, d_model, nhead, d_ff, num_layers, dropout_rate, deepfeat_sz, nb_feats, nb_demo, classes):
        super(CTN, self).__init__()

        self.encoder = nn.Sequential(  # downsampling factor = 20
            nn.Conv1d(1, 128, kernel_size=14, stride=3, padding=2, bias=False),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Conv1d(128, 256, kernel_size=14, stride=3, padding=0, bias=False),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Conv1d(256, d_model, kernel_size=10, stride=2, padding=0, bias=False),
            nn.BatchNorm1d(d_model),
            nn.ReLU(inplace=True),
            nn.Conv1d(d_model, d_model, kernel_size=10, stride=2, padding=0, bias=False),
            nn.BatchNorm1d(d_model),
            nn.ReLU(inplace=True),
            nn.Conv1d(d_model, d_model, kernel_size=10, stride=1, padding=0, bias=False),
            nn.BatchNorm1d(d_model),
            nn.ReLU(inplace=True),
            nn.Conv1d(d_model, d_model, kernel_size=10, stride=1, padding=0, bias=False),
            nn.BatchNorm1d(d_model),
            nn.ReLU(inplace=True)
        )
        self.transformer = Transformer(d_model, nhead, d_ff, num_layers, dropout=0.1)
        self.fc1 = nn.Linear(d_model, deepfeat_sz)
        # self.fc2 = nn.Linear(deepfeat_sz+nb_feats+nb_demo, len(classes))
        self.fc2 = nn.Linear(deepfeat_sz, len(classes))
        self.dropout = nn.Dropout(dropout_rate)

        def _weights_init(m):
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # self.apply(_weights_init)

    def forward(self, x, wide_feats):
        z = self.encoder(x)  # encoded sequence is batch_sz x nb_ch x seq_len
        out = self.transformer(z)  # transformer output is batch_sz x d_model
        out = self.dropout(F.relu(self.fc1(out)))
        # out = self.fc2(torch.cat([wide_feats, out], dim=1))
        out = self.fc2(out)
        return out



## Model Loading Methods

In [4]:
def load_best_model(model_loc, deepfeat_sz, remove_last_layer=False):
    model = CTN(d_model, nhead, d_ff, num_layers, dropout_rate, deepfeat_sz, nb_feats, nb_demo, classes).to(device)
    checkpoint = torch.load(model_loc, map_location=torch.device('cpu'))
    model = torch.nn.DataParallel(model)

    model.load_state_dict(checkpoint['model_state_dict'])
    print('Loading best model: best_loss', checkpoint['best_loss'], 'best_auroc', checkpoint['best_auroc'],
          'at epoch',
          checkpoint['epoch'])

    if remove_last_layer:
        model = torch.nn.Sequential(*(list(list(model.children())[0].children())[:-2]))
    return model


## Load the pre-existing Transformer model

In [5]:

thrs = np.loadtxt("/deep/u/tomjin/aihc-aut20-selfecg/prna/outputs-wide-64-15sec-bs64/saved_models/ctn/fold_1/thrs.txt")
model = load_best_model("/deep/u/tomjin/aihc-aut20-selfecg/prna/outputs-wide-64-15sec-bs64/saved_models/ctn/fold_1/ctn.tar", 64)
model.eval()

Loading best model: best_loss 0.10535395583685707 best_auroc tensor(0.8580) at epoch 31


DataParallel(
  (module): CTN(
    (encoder): Sequential(
      (0): Conv1d(1, 128, kernel_size=(14,), stride=(3,), padding=(2,), bias=False)
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv1d(128, 256, kernel_size=(14,), stride=(3,), bias=False)
      (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv1d(256, 256, kernel_size=(10,), stride=(2,), bias=False)
      (7): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU(inplace=True)
      (9): Conv1d(256, 256, kernel_size=(10,), stride=(2,), bias=False)
      (10): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (11): ReLU(inplace=True)
      (12): Conv1d(256, 256, kernel_size=(10,), stride=(1,), bias=False)
      (13): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(model)

13623384

In [7]:
list(list(model.children())[0].children())[0]

Sequential(
  (0): Conv1d(1, 128, kernel_size=(14,), stride=(3,), padding=(2,), bias=False)
  (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Conv1d(128, 256, kernel_size=(14,), stride=(3,), bias=False)
  (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): Conv1d(256, 256, kernel_size=(10,), stride=(2,), bias=False)
  (7): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU(inplace=True)
  (9): Conv1d(256, 256, kernel_size=(10,), stride=(2,), bias=False)
  (10): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): ReLU(inplace=True)
  (12): Conv1d(256, 256, kernel_size=(10,), stride=(1,), bias=False)
  (13): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): ReLU(inplace=True)
  (15): Conv1d(256, 256, kernel_size=(10,), stride=(1,), bias

In [8]:
input = torch.zeros((10, 1, 15 * 500)).cuda()
model(input, None).shape # bs, 1, ECG

torch.Size([10, 24])

## Dataloaders

In [9]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pywt
import wfdb
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from wfdb import processing

from scipy.io import loadmat
from tqdm import tqdm_notebook
from scipy.signal import decimate, resample
from biosppy.signals import ecg
from biosppy.signals.tools import filter_signal

import random

class ECGWindowPaddingDataset(Dataset):
    def __init__(self, df, window_length, nb_windows, src_path, cls=classes, lead=2):
        ''' Return randome window length segments from ecg signal, pad if window is too large
            df: trn_df, val_df or tst_df
            window: ecg window length e.g 2500 (5 seconds)
            nb_windows: number of windows to sample from record
        '''
        self.df = df
        self.window_length = window_length
        self.nb_windows = nb_windows
        self.src_path = src_path
        self.classes = cls
        self.lead = lead

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Get data
        row = self.df.iloc[idx]
        filename = str(self.src_path + "/" + (row.Patient + '.hea'))
        data, hdr = load_challenge_data(filename, self.lead, self.window_length)

        # Apply band pass filter
        data = apply_filter(data, [3, 45])
        data = normalize(data)

        lbl = row[self.classes].values.astype(int)
        
        # Add just enough padding to allow window
        seq_len = len(data)
        pad = np.abs(np.min(seq_len - self.window_length, 0))
        if pad > 0:
            data = np.pad(data, (0, pad+1))
            seq_len = len(data) # get the new length of the ecg sequence

        starts = np.random.randint(seq_len - self.window_length + 1, size=self.nb_windows) # get start indices of ecg segment    
        
        ecg_segs = []
        for start in starts:
            a = data[start:start+self.window_length]
            ecg_segs.append([a])
        ecg_segs = np.array(ecg_segs)
        return ecg_segs, lbl, hdr, filename


def load_challenge_data(header_file, lead=2, window_length=7500):
    with open(header_file, 'r') as f:
        header = f.readlines()    
    sampling_rate = int(header[0].split()[2])    
    mat_file = header_file.replace('.hea', '.mat')
    try:
        x = loadmat(mat_file)
        recording = np.asarray(x['val'], dtype=np.float64)[lead-1, :] # To use Lead III data, pass in lead=3
    except Exception as error:
        print(error)
        print(f"Found mat_file that couldn't be loaded: {mat_file}")
        recording = np.zeros((window_length * 500, ))
    
    # Standardize sampling rate
    if sampling_rate > 500:
        recording = decimate(recording, int(sampling_rate / 500))
    elif sampling_rate < 500:
        recording = resample(recording, int(len(recording) * (500 / sampling_rate)))
    
    return recording, header


## Show Grad-CAM

In [10]:
import cv2
import numpy as np
import torch
import ttach as tta
from pytorch_grad_cam.activations_and_gradients import ActivationsAndGradients
from pytorch_grad_cam.utils.svd_on_activations import get_2d_projection


class BaseCAM1D:
    def __init__(self, 
                 model, 
                 target_layer,
                 use_cuda=False,
                 reshape_transform=None):
        self.model = model.eval()
        self.target_layer = target_layer
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()
        self.reshape_transform = reshape_transform
        self.activations_and_grads = ActivationsAndGradients(self.model, 
            target_layer, reshape_transform)

    def forward(self, input_img):
        return self.model(input_img, None)

    def get_cam_weights(self,
                        input_tensor,
                        target_category,
                        activations,
                        grads):
        raise Exception("Not Implemented")

    def get_loss(self, output, target_category):
        loss = 0
        for i in range(len(target_category)):
            loss = loss + output[i, target_category[i]]
        return loss

    def get_cam_image(self,
                      input_tensor,
                      target_category,
                      activations,
                      grads,
                      eigen_smooth=False):
        weights = self.get_cam_weights(input_tensor, target_category, activations, grads)
        weighted_activations = weights[:, :, None] * activations
        if eigen_smooth:
#             cam = get_2d_projection(weighted_activations)
            raise Exception("Not translated to 1D")
        else:
            cam = weighted_activations.sum(axis=1)
        return cam

    def forward(self, input_tensor, target_category=None, eigen_smooth=False):

        if self.cuda:
            input_tensor = input_tensor.cuda()

        output = self.activations_and_grads(input_tensor)

        if type(target_category) is int:
            target_category = [target_category] * input_tensor.size(0)

        if target_category is None:
            target_category = np.argmax(output.cpu().data.numpy(), axis=-1)
        else:
            assert(len(target_category) == input_tensor.size(0))

        self.model.zero_grad()
        loss = self.get_loss(output, target_category)
        loss.backward(retain_graph=True)

        activations = self.activations_and_grads.activations[-1].cpu().data.numpy()
        grads = self.activations_and_grads.gradients[-1].cpu().data.numpy()

        cam = self.get_cam_image(input_tensor, target_category, 
            activations, grads, eigen_smooth)

        cam = np.maximum(cam, 0)

        result = []
        for img in cam:
#             plt.plot(np.squeeze(img))
#             plt.show()
            
            img = cv2.resize(img, (1, 7500))
            img = img - np.min(img)
            img = img / np.max(img)
#             plt.plot(np.squeeze(img))
#             plt.show()
            result.append(img)
        result = np.float32(result)
        return result

    def __call__(self,
                 input_tensor,
                 target_category=None,
                 aug_smooth=False,
                 eigen_smooth=False):

        return self.forward(input_tensor,
            target_category, eigen_smooth)

In [11]:
class ActivationsAndGradients:
    """ Class for extracting activations and
    registering gradients from targetted intermediate layers """

    def __init__(self, model, target_layer, reshape_transform):
        self.model = model
        self.gradients = []
        self.activations = []
        self.reshape_transform = reshape_transform

        target_layer.register_forward_hook(self.save_activation)
        target_layer.register_backward_hook(self.save_gradient)

    def save_activation(self, module, input, output):
        activation = output
        if self.reshape_transform is not None:
            activation = self.reshape_transform(activation)
        self.activations.append(activation.cpu().detach())

    def save_gradient(self, module, grad_input, grad_output):
        # Gradients are computed in reverse order
        grad = grad_output[0]
        if self.reshape_transform is not None:
            grad = self.reshape_transform(grad)
        self.gradients = [grad.cpu().detach()] + self.gradients

    def __call__(self, x):
        self.gradients = []
        self.activations = []        
        return self.model(x, None)

In [12]:
import cv2
import numpy as np
import torch
from pytorch_grad_cam.base_cam import BaseCAM

class GradCAM1D(BaseCAM1D):
    def __init__(self, model, target_layer, use_cuda=False, 
        reshape_transform=None):
        super(GradCAM1D, self).__init__(model, target_layer, use_cuda, reshape_transform)

    def get_cam_weights(self,
                        input_tensor,
                        target_category,
                        activations,
                        grads):
#         print(f"gradsshape: {grads.shape}")
        output_mean = np.mean(grads, axis=2)
#         print(f"output_mean = {output_mean.shape}")
#         print(f"output_mean: {output_mean}")
        return output_mean

In [13]:
# Construct the CAM object once, and then re-use it on many images:
cam = GradCAM1D(model=model, target_layer=list(list(model.children())[0].children())[0], use_cuda=True)


## Create Full Images

In [15]:
def plot_ecgs(correct_df, df_offsets, patient_ids=None, rows=30, show_labels=True):
    
    fig, ax = plt.subplots(rows, 3, figsize=(15, (45 / 30) * rows))
    fig.tight_layout()

    ind = 0
    for i, row in tqdm(correct_df.iterrows()):
        patient_id = int(row["patient_id"])
        if patient_ids is not None and patient_id not in patient_ids:
            continue

        label = int(row["outcome"])
        actual = int(float(row["preds"]) >= best_thresh)
        actual_row = ind

        df_offsets_sub = df_offsets[df_offsets["record_name"] == patient_id]

        start_offset_1 = int(df_offsets_sub.index[0])
        start_offset_2 = int(df_offsets_sub.index[1])
        start_offset_3 = int(df_offsets_sub.index[2])
        ecg1 = ecg[start_offset_1, :]
        ecg2 = ecg[start_offset_2, :]
        ecg3 = ecg[start_offset_3, :]

        ecg1_torch = torch.from_numpy(ecg1)
        ecg1_torch = torch.reshape(ecg1_torch, (1, 1, -1))
        ecg1_torch = ecg1_torch.float().cuda()
        ecg2_torch = torch.from_numpy(ecg2)
        ecg2_torch = torch.reshape(ecg2_torch, (1, 1, -1))
        ecg2_torch = ecg2_torch.float().cuda()
        ecg3_torch = torch.from_numpy(ecg3)
        ecg3_torch = torch.reshape(ecg3_torch, (1, 1, -1))
        ecg3_torch = ecg3_torch.float().cuda()

        grayscale_cam_1 = cam(input_tensor=ecg1_torch)
        grayscale_cam_2 = cam(input_tensor=ecg2_torch)
        grayscale_cam_3 = cam(input_tensor=ecg3_torch)

        grayscale_cam_1 = np.squeeze(grayscale_cam_1)[:500*3]
        grayscale_cam_2 = np.squeeze(grayscale_cam_2)[:500*3]
        grayscale_cam_3 = np.squeeze(grayscale_cam_3)[:500*3]

        ecg1 = ecg1[:500*3]
        ecg2 = ecg2[:500*3]
        ecg3 = ecg3[:500*3]

        ax[actual_row, 0].plot(ecg1)
        ax[actual_row, 0].fill_between(range(500*3), ecg1 - grayscale_cam_1, ecg1 + grayscale_cam_1, color='#2e74e6', alpha=0.2)
        ax[actual_row, 0].set_ylim([-1, 1])
        ax[actual_row, 0].set_ylabel(patient_id)
        ax[actual_row, 0].set_xlim([0, 500*3])
        ax[actual_row, 0].set_xticks([])
        ax[actual_row, 0].set_xticks([], minor=True)
        ax[actual_row, 0].set_yticks([])
        ax[actual_row, 0].set_yticks([], minor=True)

        ax[actual_row, 1].plot(ecg2[:500*3])
        ax[actual_row, 1].fill_between(range(500*3), ecg2 - grayscale_cam_2, ecg2 + grayscale_cam_2, color='#2e74e6', alpha=0.2)
        ax[actual_row, 1].set_ylim([-1, 1])
        ax[actual_row, 1].set_xlim([0, 500*3])
        if show_labels:
            ax[actual_row, 1].set_title(f"Predicted={actual} | Actual={label}")
        ax[actual_row, 1].set_xticks([])
        ax[actual_row, 1].set_xticks([], minor=True)
        ax[actual_row, 1].set_yticks([])
        ax[actual_row, 1].set_yticks([], minor=True)

        ax[actual_row, 2].plot(ecg3[:500*3])
        ax[actual_row, 2].fill_between(range(500*3), ecg3 - grayscale_cam_3, ecg3 + grayscale_cam_3, color='#2e74e6', alpha=0.2)
        ax[actual_row, 2].set_ylim([-1, 1])
        ax[actual_row, 2].set_xlim([0, 500*3])
        ax[actual_row, 2].set_xticks([])
        ax[actual_row, 2].set_xticks([], minor=True)
        ax[actual_row, 2].set_yticks([])
        ax[actual_row, 2].set_yticks([], minor=True)

        if ind >= (rows - 1):
            break
        ind += 1

    plt.show()

    

In [16]:
ecg = np.load(f"/deep/group/ed-monitor/patient_data_v9/waveforms/15sec-500hz-1norm-10wpp/II/waveforms.dat.npy")


In [ ]:
df_offsets = pd.read_csv(f"/deep/group/ed-monitor/patient_data_v9/waveforms/15sec-500hz-1norm-10wpp/II/summary.csv")
df_offsets.head(3)

In [18]:
# Find average threshold on validation set
# See: Threshold notebook

best_thresh = 0.096377

In [19]:
# Apply threshold to test (all patients)

df = pd.read_csv(f"/deep/group/ed-monitor/patient_data_v9/predictions/15sec-500hz-1norm-10wpp/II/final-transformer-64/waveform-only/test.csv")
df_cohort = pd.read_csv(f"/deep/group/ed-monitor/patient_data_v9/consolidated.filtered.test.txt", sep="\t")
df = pd.merge(df, df_cohort, on="patient_id")
testy = df["actual"]
yhat = df["preds"]

preds_t = (df["preds"] >= best_thresh).to_numpy()
accuracy = sum(preds_t == df["actual"]) / len(df["actual"])
print(f"Average accuracies = {accuracy}")


Average accuracies = 0.6998525073746312
